# YOLO Nail Detection Model Training

This notebook trains a custom YOLO model specifically for nail detection in hand images.

## Prerequisites

Before training, you need:
1. **Annotated dataset** with bounding boxes around nails in hand images
2. **YOLO format annotations** (`.txt` files with normalized coordinates)
3. **Organized directory structure** (see Data Preparation section)

## Data Requirements

- **Images**: Hand images containing visible nails
- **Annotations**: One `.txt` file per image with bounding box coordinates
- **Format**: YOLO format (normalized: class_id x_center y_center width height)


In [ ]:
# Install required packages
%pip install ultralytics opencv-python pillow matplotlib


## Step 1: Data Preparation

### Directory Structure

Your dataset should be organized as follows:

```
yolo_nail_dataset/
├── train/
│   ├── images/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── labels/
│       ├── image1.txt
│       ├── image2.txt
│       └── ...
├── val/
│   ├── images/
│   │   ├── val1.jpg
│   │   └── ...
│   └── labels/
│       ├── val1.txt
│       └── ...
└── test/  (optional)
    ├── images/
    └── labels/
```

### YOLO Annotation Format

Each `.txt` file should contain one line per nail detection:
```
class_id x_center y_center width height
```

Where:
- `class_id`: 0 (for nail class - single class detection)
- All coordinates are **normalized** (0.0 to 1.0)
- `x_center, y_center`: Center of bounding box (normalized)
- `width, height`: Width and height of bounding box (normalized)

**Example annotation** (`image1.txt`):
```
0 0.5 0.3 0.1 0.15
0 0.7 0.3 0.1 0.15
```
This means 2 nails detected in the image.

### Annotation Tools

You can use tools like:
- **LabelImg**: https://github.com/HumanSignal/labelImg
- **Roboflow**: https://roboflow.com/
- **CVAT**: https://cvat.org/
- **Label Studio**: https://labelstud.io/


In [ ]:
import os
import yaml
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Configuration
DATASET_DIR = "yolo_nail_dataset"  # Change this to your dataset path
MODEL_SIZE = "n"  # Options: n (nano), s (small), m (medium), l (large), x (xlarge)
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 16
PATIENCE = 50  # Early stopping patience

# Output directory
OUTPUT_DIR = "runs/detect/nail-detector"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Configuration loaded!")
print(f"Model size: YOLOv8{MODEL_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Image size: {IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")


## Step 2: Create Dataset Configuration

We need to create a `dataset.yaml` file that YOLO uses to locate the training data.


In [ ]:
# Create dataset.yaml configuration file
dataset_config = {
    'path': os.path.abspath(DATASET_DIR),  # Absolute path to dataset
    'train': 'train/images',  # Relative to 'path'
    'val': 'val/images',      # Relative to 'path'
    'test': 'test/images' if os.path.exists(os.path.join(DATASET_DIR, 'test')) else None,  # Optional
    
    # Class names
    'names': {
        0: 'nail'
    },
    
    # Number of classes
    'nc': 1
}

# Save configuration
yaml_path = os.path.join(DATASET_DIR, 'dataset.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False, sort_keys=False)

print(f"Dataset configuration saved to: {yaml_path}")
print("\nConfiguration:")
print(yaml.dump(dataset_config, default_flow_style=False))


## Step 3: Verify Dataset Structure

Let's verify that your dataset is properly organized before training.


In [ ]:
def verify_dataset(dataset_dir):
    """Verify dataset structure and annotations"""
    issues = []
    warnings = []
    
    # Check directory structure
    train_img_dir = os.path.join(dataset_dir, 'train', 'images')
    train_label_dir = os.path.join(dataset_dir, 'train', 'labels')
    val_img_dir = os.path.join(dataset_dir, 'val', 'images')
    val_label_dir = os.path.join(dataset_dir, 'val', 'labels')
    
    # Check if directories exist
    for dir_path, name in [(train_img_dir, 'train/images'), 
                           (train_label_dir, 'train/labels'),
                           (val_img_dir, 'val/images'),
                           (val_label_dir, 'val/labels')]:
        if not os.path.exists(dir_path):
            issues.append(f"Missing directory: {name}")
        else:
            print(f"✓ Found: {name}")
    
    if issues:
        print("\n❌ Issues found:")
        for issue in issues:
            print(f"  - {issue}")
        return False
    
    # Count files
    train_images = [f for f in os.listdir(train_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))] if os.path.exists(train_img_dir) else []
    train_labels = [f for f in os.listdir(train_label_dir) if f.endswith('.txt')] if os.path.exists(train_label_dir) else []
    
    val_images = [f for f in os.listdir(val_img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))] if os.path.exists(val_img_dir) else []
    val_labels = [f for f in os.listdir(val_label_dir) if f.endswith('.txt')] if os.path.exists(val_label_dir) else []
    
    print(f"\n📊 Dataset Statistics:")
    print(f"  Training images: {len(train_images)}")
    print(f"  Training labels: {len(train_labels)}")
    print(f"  Validation images: {len(val_images)}")
    print(f"  Validation labels: {len(val_labels)}")
    
    # Check for matching files
    train_img_names = {os.path.splitext(f)[0] for f in train_images}
    train_label_names = {os.path.splitext(f)[0] for f in train_labels}
    
    missing_labels = train_img_names - train_label_names
    missing_images = train_label_names - train_img_names
    
    if missing_labels:
        warnings.append(f"⚠️  {len(missing_labels)} training images without labels")
    if missing_images:
        warnings.append(f"⚠️  {len(missing_images)} training labels without images")
    
    if warnings:
        print("\n⚠️  Warnings:")
        for warning in warnings:
            print(f"  - {warning}")
    
    # Validate annotation format
    if train_labels:
        sample_label = os.path.join(train_label_dir, train_labels[0])
        try:
            with open(sample_label, 'r') as f:
                lines = f.readlines()
                if lines:
                    parts = lines[0].strip().split()
                    if len(parts) == 5:
                        class_id, x, y, w, h = map(float, parts)
                        if 0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1:
                            print(f"\n✓ Sample annotation format is correct: {train_labels[0]}")
                        else:
                            issues.append("Annotation coordinates not normalized (should be 0-1)")
                    else:
                        issues.append(f"Invalid annotation format in {train_labels[0]} (expected 5 values)")
        except Exception as e:
            issues.append(f"Error reading annotation: {e}")
    
    if not issues:
        print("\n✅ Dataset structure looks good!")
        return True
    else:
        print("\n❌ Please fix the issues above before training.")
        return False

# Verify dataset
if os.path.exists(DATASET_DIR):
    verify_dataset(DATASET_DIR)
else:
    print(f"❌ Dataset directory not found: {DATASET_DIR}")
    print("\nPlease create the dataset directory with the following structure:")
    print("yolo_nail_dataset/")
    print("  ├── train/")
    print("  │   ├── images/")
    print("  │   └── labels/")
    print("  └── val/")
    print("      ├── images/")
    print("      └── labels/")


In [ ]:
# Load pretrained YOLOv8 model
model_name = f"yolov8{MODEL_SIZE}.pt"
print(f"Loading pretrained model: {model_name}")

model = YOLO(model_name)
print("✓ Model loaded successfully!")

# Display model info
print(f"\nModel architecture: YOLOv8{MODEL_SIZE}")
print(f"Parameters: {sum(p.numel() for p in model.model.parameters()):,}")


## Step 5: Train the Model

This will start the training process. Training may take a while depending on:
- Number of images
- Model size (n/s/m/l/x)
- Hardware (CPU/GPU)
- Number of epochs


In [ ]:
# Training configuration
train_args = {
    'data': yaml_path,           # Path to dataset.yaml
    'epochs': EPOCHS,            # Number of training epochs
    'imgsz': IMG_SIZE,           # Image size
    'batch': BATCH_SIZE,         # Batch size
    'name': 'nail-detector',     # Project name
    'patience': PATIENCE,        # Early stopping patience
    'save': True,                # Save checkpoints
    'save_period': 10,           # Save checkpoint every N epochs
    'device': 0,                 # GPU device (0 for first GPU, 'cpu' for CPU)
    'workers': 8,                # Number of data loading workers
    'project': 'runs/detect',    # Project directory
    'exist_ok': True,            # Overwrite existing project
    'pretrained': True,          # Use pretrained weights
    'optimizer': 'AdamW',        # Optimizer
    'lr0': 0.01,                 # Initial learning rate
    'lrf': 0.01,                 # Final learning rate (lr0 * lrf)
    'momentum': 0.937,           # SGD momentum/Adam beta1
    'weight_decay': 0.0005,      # Weight decay
    'warmup_epochs': 3.0,        # Warmup epochs
    'warmup_momentum': 0.8,      # Warmup initial momentum
    'warmup_bias_lr': 0.1,       # Warmup initial bias lr
    'box': 7.5,                  # Box loss gain
    'cls': 0.5,                  # Class loss gain
    'dfl': 1.5,                  # DFL loss gain
    'hsv_h': 0.015,              # Image HSV-Hue augmentation
    'hsv_s': 0.7,                # Image HSV-Saturation augmentation
    'hsv_v': 0.4,                # Image HSV-Value augmentation
    'degrees': 0.0,              # Image rotation (+/- deg)
    'translate': 0.1,           # Image translation (+/- fraction)
    'scale': 0.5,                # Image scale (+/- gain)
    'shear': 0.0,                # Image shear (+/- deg)
    'perspective': 0.0,          # Image perspective (+/- fraction)
    'flipud': 0.0,               # Image flip up-down (probability)
    'fliplr': 0.5,               # Image flip left-right (probability)
    'mosaic': 1.0,               # Image mosaic (probability)
    'mixup': 0.0,                # Image mixup (probability)
    'copy_paste': 0.0,           # Segment copy-paste (probability)
}

print("Starting training...")
print(f"Training for {EPOCHS} epochs")
print(f"Dataset: {yaml_path}")
print(f"Output: {OUTPUT_DIR}")
print("\n" + "="*60)

# Start training
results = model.train(**train_args)

print("\n" + "="*60)
print("✅ Training completed!")
print(f"Best model saved to: {results.save_dir}/weights/best.pt")
print(f"Last model saved to: {results.save_dir}/weights/last.pt")


## Step 6: Evaluate the Model

Let's evaluate the trained model on the validation set.


In [ ]:
# Load the best model
best_model_path = os.path.join(results.save_dir, 'weights', 'best.pt')
print(f"Loading best model from: {best_model_path}")

best_model = YOLO(best_model_path)

# Evaluate on validation set
metrics = best_model.val(data=yaml_path, imgsz=IMG_SIZE)

print("\n" + "="*60)
print("📊 Validation Results:")
print("="*60)
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")


## Step 7: Test on Sample Images

Test the trained model on some sample images to visualize detections.


In [ ]:
def visualize_predictions(model, image_path, conf_threshold=0.25, save_path=None):
    """Visualize model predictions on an image"""
    # Run inference
    results = model(image_path, conf=conf_threshold, imgsz=IMG_SIZE)
    
    # Get the first result
    result = results[0]
    
    # Plot results
    annotated_img = result.plot()
    
    # Convert BGR to RGB for matplotlib
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_img_rgb)
    plt.axis('off')
    plt.title(f'Nail Detection Results (conf ≥ {conf_threshold})')
    plt.tight_layout()
    
    if save_path:
        cv2.imwrite(save_path, annotated_img)
        print(f"Saved visualization to: {save_path}")
    
    plt.show()
    
    # Print detection info
    if len(result.boxes) > 0:
        print(f"\nDetected {len(result.boxes)} nail(s):")
        for i, box in enumerate(result.boxes):
            conf = float(box.conf[0])
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            print(f"  Nail {i+1}: confidence={conf:.3f}, bbox=({x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f})")
    else:
        print("\nNo nails detected.")
    
    return result

# Test on validation images (if available)
val_img_dir = os.path.join(DATASET_DIR, 'val', 'images')
if os.path.exists(val_img_dir):
    val_images = [f for f in os.listdir(val_img_dir) 
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if val_images:
        # Test on first validation image
        test_image = os.path.join(val_img_dir, val_images[0])
        print(f"Testing on: {test_image}")
        visualize_predictions(best_model, test_image, conf_threshold=0.25)
    else:
        print("No validation images found for testing.")
else:
    print("Validation image directory not found.")


## Step 8: Export and Save Model

Save the trained model to a location where the Flask app can use it.


In [ ]:
# Copy best model to models directory for use in Flask app
import shutil

models_dir = "models"
os.makedirs(models_dir, exist_ok=True)

# Copy best model
best_model_dest = os.path.join(models_dir, "yolo_nail_detector_best.pt")
shutil.copy2(best_model_path, best_model_dest)

print(f"✅ Best model copied to: {best_model_dest}")
print(f"\nTo use this model in your Flask app, update app.py:")
print(f"  nail_detector = NailDetector(model_path='{best_model_dest}')")

# Also export to ONNX format (optional, for deployment)
try:
    onnx_path = os.path.join(models_dir, "yolo_nail_detector.onnx")
    best_model.export(format='onnx', imgsz=IMG_SIZE)
    print(f"\n✅ Model exported to ONNX format")
except Exception as e:
    print(f"\n⚠️  ONNX export failed: {e}")


## Step 9: Training Summary

Review the training results and metrics.


In [ ]:
# Display training curves
results_dir = results.save_dir
results_csv = os.path.join(results_dir, 'results.csv')

if os.path.exists(results_csv):
    import pandas as pd
    
    df = pd.read_csv(results_csv)
    
    # Plot training curves
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='blue')
    axes[0, 0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', color='red')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Box Loss')
    axes[0, 0].set_title('Box Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(df['epoch'], df['train/cls_loss'], label='Train Class Loss', color='blue')
    axes[0, 1].plot(df['epoch'], df['val/cls_loss'], label='Val Class Loss', color='red')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Class Loss')
    axes[0, 1].set_title('Class Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # mAP curves
    axes[1, 0].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50', color='green')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('mAP50')
    axes[1, 0].set_title('Mean Average Precision (mAP50)')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95', color='purple')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('mAP50-95')
    axes[1, 1].set_title('Mean Average Precision (mAP50-95)')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print best metrics
    best_epoch = df['metrics/mAP50(B)'].idxmax()
    print("\n" + "="*60)
    print("📈 Best Training Metrics:")
    print("="*60)
    print(f"Best Epoch: {int(df.loc[best_epoch, 'epoch'])}")
    print(f"Best mAP50: {df.loc[best_epoch, 'metrics/mAP50(B)']:.4f}")
    print(f"Best mAP50-95: {df.loc[best_epoch, 'metrics/mAP50-95(B)']:.4f}")
    print(f"Best Precision: {df.loc[best_epoch, 'metrics/precision(B)']:.4f}")
    print(f"Best Recall: {df.loc[best_epoch, 'metrics/recall(B)']:.4f}")
else:
    print("Results CSV not found. Training curves may not be available.")
